In [122]:
%matplotlib inline

In [123]:
import pandas as pd
import numpy as np

# Importing the data from sources

## Inequality measures Deininger and Squire

Not sure whether the good quality criteria in Forbes deals with the variable Quality or Q. In order to be confident of our data, we impose the two conditions. 
This leads us to 679 points which is very close to the 682 points of the paper. 

We also add 6.6 we the data is based on expenditure and not on income (on the same cell to be sure that it is not done twice. 

In [124]:
DS = pd.read_excel("Deininger_and Squire.XLS", usecols=["Code", "Quality", "Year", "Gini", "Q", 'Inc'])
DS = DS.query("Q == 'good' and Quality == 'accept'")
DS.drop_duplicates(["Code", "Year"], inplace=True)
DS['Gini'].astype(float, inplace=True)

DS.loc[DS['Inc']=='E', 'Gini'] += 6.6

DS = DS[["Code", "Year", "Gini"]]
DS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 2631
Data columns (total 3 columns):
Code    679 non-null object
Year    679 non-null int64
Gini    679 non-null object
dtypes: int64(1), object(2)
memory usage: 21.2+ KB


## Educational data : Barro-Lee

We take the data directly from the BarroLee website. Cf list of name variable name.

In [125]:
male_educ = pd.read_csv("male_attainment_25_BarroLee.csv", usecols=["year", "WBcode", "yr_sch_sec"])
female_educ = pd.read_csv("female_attainment_25_BarroLee.csv", usecols=["year", "WBcode", "yr_sch_sec"])

In [126]:
print male_educ.info()
print female_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 1897
Data columns (total 3 columns):
year          1898 non-null int64
yr_sch_sec    1898 non-null float64
WBcode        1898 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 59.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1898 entries, 0 to 1897
Data columns (total 3 columns):
year          1898 non-null int64
yr_sch_sec    1898 non-null float64
WBcode        1898 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 59.3+ KB
None


## Income

We downloaded the data from data.worldbank. The name of the variable has changed from GNP to GNI we should investigate to decide whether or not the differences are big. 
We secretly hope that the data before 1995 hasn't been changed and consequently that the new name of the variable affects nothing for our study. 

In [148]:
inc = pd.read_csv("GNI_per_capita_WB.csv", skiprows=4)
del inc["Country Name"], inc['Indicator Code'], inc['Indicator Name']
inc.set_index("Country Code", inplace=True)
inc = inc.stack().reset_index()
inc.columns = ["code", "year", "GNI_PC"]
print inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9131 entries, 0 to 9130
Data columns (total 3 columns):
code      9131 non-null object
year      9131 non-null object
GNI_PC    9131 non-null float64
dtypes: float64(1), object(2)
memory usage: 285.3+ KB
None


## Price Level of Investment

Extracted from the Penn World Table 5.6 (as in Forbes). The name of the variable is PI (cf. documentation column number [15]). 

We now convert the Country name to the corresponding code using our good old dictionary. 

In [140]:
PPPI = pd.read_excel("pwt56_forweb.xls", usecols=["Country", "Year", "PI"]).dropna(subset=["PI"])
country_dict = pd.read_csv("../data_source/country_code_list.csv")

country_dict.loc[:, 'country'] = country_dict['country'].apply(lambda x: x.upper())
country_dict.set_index('country', inplace=True)

PPPI.replace("CAPE VERDE IS.", "CABO VERDE", inplace=True)
PPPI.replace("CENTRAL AFR.R.", "CENTRAL AFRICAN REPUBLIC", inplace=True)
PPPI.replace("GUINEA-BISS", "GUINEA-BISSAU", inplace=True)

PPPI = PPPI.query("Country != 'REUNION'")

PPPI = PPPI.query("Country != 'ZAIRE'")

PPPI.replace("DOMINICAN REP.", "DOMINICAN REPUBLIC", inplace=True)
PPPI.replace("ST.KITTS&NEVIS", "SAINT KITTS AND NEVIS", inplace=True)
PPPI.replace("ST.LUCIA", "SAINT LUCIA", inplace=True)
PPPI.replace("TRINIDAD&TOBAGO", "TRINIDAD AND TOBAGO", inplace=True)
PPPI.replace("U.S.A.", "UNITED STATES OF AMERICA", inplace=True)
PPPI.replace("KOREA, REP.", "SOUTH KOREA", inplace=True)
PPPI.replace("SYRIA", "SYRIAN ARAB REPUBLIC", inplace=True)
PPPI.replace("UNITED ARAB E.", "UNITED ARAB EMIRATES", inplace=True)
PPPI.replace("YEMEN", "REPUBLIC OF YEMEN", inplace=True)

PPPI = PPPI.query("Country != 'GERMANY, EAST'")

PPPI.replace("GERMANY, WEST", "GERMANY", inplace=True)
PPPI.replace("U.K.", "UNITED KINGDOM", inplace=True)
PPPI.replace("U.S.S.R.", "RUSSIAN FEDERATION", inplace=True)
PPPI.replace("PAPUA N.GUINEA", "PAPUA NEW GUINEA", inplace=True)
PPPI.replace("SOLOMON IS.", "SOLOMON ISLANDS", inplace=True)
PPPI.replace("ST.VINCENT&GRE", "SAINT VINCENT AND THE GRENADINES", inplace=True)
PPPI.replace("WESTERN SAMOA", "SAMOA", inplace=True)


PPPI['code'] = PPPI['Country'].apply(lambda x: country_dict.loc[x])
PPPI = PPPI[['code', 'Year', 'PI']]

In [141]:
PPPI.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4950 entries, 10 to 6533
Data columns (total 3 columns):
code    4950 non-null object
Year    4950 non-null int64
PI      4950 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 154.7+ KB


# Gathering the data in one frame

The first step is to remove rename the columns in preparation of the merging of the databases. 
We also remove all the duplicates in order to avoid the problems during the merging. 

### Drop duplicates and normalize the data from different sources

In [214]:
# rename the columns (in order to merge)
DS.columns = ["code", "year", "gini"]
PPPI.columns = ["code", "year", "PPPI"]
male_educ.columns = ["year", "sch_male", "code"]
female_educ.columns = ["year", "sch_female", "code"]

# drop duplicates in order to avoid problems during the merging
print "Income duplicates :", inc.duplicated(subset=['code', 'year']).sum()
print "Inequality duplicates :", DS.duplicated(subset=['code', 'year']).sum()
print "PPPI duplicates :", PPPI.duplicated(subset=['code', 'year']).sum()
print "Male educ duplicates :", male_educ.duplicated(subset=['code', 'year']).sum()
print "Female Educ duplicates :", female_educ.duplicated(subset=['code', 'year']).sum()

# normalize the data types
DS.loc[:, 'year'] = DS['year'].astype(int)
DS.loc[:, 'code'] = DS['code'].astype(str)
DS.loc[:, 'gini'] = DS['gini'].astype(float)

PPPI.loc[:, 'year'] = PPPI['year'].astype(int)
PPPI.loc[:, 'code'] = PPPI['code'].astype(str)
PPPI.loc[:, 'PPPI'] = PPPI['PPPI'].astype(float)

male_educ.loc[:, 'year'] = male_educ['year'].astype(int)
male_educ.loc[:, 'code'] = male_educ['code'].astype(str)
male_educ.loc[:, 'sch_male'] = male_educ['sch_male'].astype(float)

female_educ.loc[:, 'year'] = female_educ['year'].astype(int)
female_educ.loc[:, 'code'] = female_educ['code'].astype(str)
female_educ.loc[:, 'sch_female'] = female_educ['sch_female'].astype(float)

inc.loc[:, 'year'] = inc['year'].astype(int)
inc.loc[:, 'code'] = inc['code'].astype(str)
inc.loc[:, 'GNI_PC'] = inc['GNI_PC'].astype(float)

Income duplicates : 0
Inequality duplicates : 0
PPPI duplicates : 0
Male educ duplicates : 0
Female Educ duplicates : 0


We now have to make sure of the concordance country codes between bases...

We found some problematic codes in the income database : codes that do not appear in our good old country code dictionary. We see that the corresponding countries are not countries or negligeable countries so we can just drop them from the database. 

In [215]:
problematic_codes = list(set(inc.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("GNI_per_capita_WB.csv", skiprows=4, usecols=['Country Name', 'Country Code'])
print country_code[country_code['Country Code'].apply(lambda x: x in problematic_codes)]
inc = inc.query("code in " + str(country_dict['code'].values.tolist()))

Empty DataFrame
Columns: [Country Name, Country Code]
Index: []


In Dieninger and Squire some codes are not used anymore in the new nomenclature so we choose to change them to the new nomenclature. 

In [216]:
problematic_codes = list(set(DS.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_excel("Deininger_and Squire.XLS", usecols=["Code", "Country"])
print country_code[country_code['Code'].apply(lambda x: x in problematic_codes)].drop_duplicates()

DS.replace("BRS", "BLR", inplace=True)
DS.replace("CSR", "CZE", inplace=True)
DS.replace("KYR", "KGZ", inplace=True)
DS.replace("LAT", "LVA", inplace=True)
DS.replace("LIT", "LTU", inplace=True)
DS.replace("MLD", "MDA", inplace=True)
DS.replace("ROM", "ROU", inplace=True)
DS.replace("SLO", "SVK", inplace=True)
DS.replace("SVA", "SVN", inplace=True)
DS.replace("SUN", "RUS", inplace=True)
DS.replace("OAN", "TWN", inplace=True)

Empty DataFrame
Columns: [Country, Code]
Index: []


In the educationnal data from Barro and Lee some codes are not used anymore in the new nomenclature so we choose to change them to the new nomenclature. 

In [217]:
problematic_codes = list(set(male_educ.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("male_attainment_25_BarroLee.csv", usecols=["WBcode", "country"])
print country_code[country_code['WBcode'].apply(lambda x: x in problematic_codes)].drop_duplicates()

male_educ.replace("ROM", "MDA", inplace=True)
male_educ.replace("SER", "SRB", inplace=True)

Empty DataFrame
Columns: [country, WBcode]
Index: []


In [218]:
problematic_codes = list(set(female_educ.query("code not in " + str(country_dict['code'].values.tolist()))['code']))
country_code = pd.read_csv("female_attainment_25_BarroLee.csv", usecols=["WBcode", "country"])
print country_code[country_code['WBcode'].apply(lambda x: x in problematic_codes)].drop_duplicates()

male_educ.replace("ROM", "MDA", inplace=True)
male_educ.replace("SER", "SRB", inplace=True)

                  country WBcode
1833  Republic of Moldova    ROM
1846               Serbia    SER


### Merging the data

In [220]:
data_frame = pd.merge(DS, PPPI, on=['code', 'year'],how='outer')
data_frame = pd.merge(data_frame, male_educ, on=['code', 'year'], how='outer')
data_frame = pd.merge(data_frame, female_educ, on=['code', 'year'], how='outer')
data_frame = pd.merge(data_frame, inc, on=['code', 'year'], how='outer')

### Resampling the data